# Figure 01 (CSV + noise CSV) — FHN Network

In [ ]:
import matplotlib.pyplot as plt
latex_ok=True
try:
 plt.rcParams.update({'text.usetex':True,'font.family':'serif','font.serif':['Computer Modern Roman'],'axes.titlesize':16,'axes.labelsize':16,'xtick.labelsize':13,'ytick.labelsize':13})
except Exception:
 latex_ok=False;plt.rcParams.update({'text.usetex':False,'font.family':'serif'})
print('LaTeX enabled:',latex_ok)

In [ ]:
from pathlib import Path
FIG_DIR=Path('figures');FIG_DIR.mkdir(exist_ok=True)
U_CSV_A='u_n_00060_c_00600_.csv'
V_CSV_A='v_n_00060_c_00600_.csv'
U_CSV_B='u_n_00140_c_00600_.csv'
V_CSV_B='v_n_00140_c_00600_.csv'
LC_PATH='limit_cycle_sample.csv'
NOISE_CSV='clean_noise_subset.csv'
NODES=50;START_OFFSET=5000
PANEL_A=dict(node=29,t0=259.11,t1=269.265,dt0=2.0,dt1=2.0,title=r'(a)',show_nullclines=True,show_limit_cycle=True,show_vectors=True)
PANEL_B=dict(node=30,t0=409.05,t1=412.01,dt0=3.0,dt1=5.0,title=r'(b)',show_nullclines=True,show_limit_cycle=True,show_vectors=True)
print('Figure dir:',FIG_DIR.resolve())

In [ ]:
figlib_code=r'''import numpy as np,matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.patches import Rectangle,FancyArrowPatch
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
A=0.7;I=0.0;NULL_ALPHA=0.5;UMIN,UMAX=-2.5,2.0;N_SAMPLES=800;LINE_WIDTH=1.0;_DASH=(1.5,3.0,6.0,3.0)
LC_PATH='limit_cycle_sample.csv';LC_ALPHA=0.5;LC_COLOR='0.35';LC_LINEWIDTH=1.0;LC_STYLE='--'
ARROW_BODY_SCALE=1.0;ARROW_HEAD_SCALE=14.0;ARROW_LW=1.6;ARROW_ALPHA=0.95;ARROW_Z=6
NODES=50;ITERATIONS=505000;P=10;ALPHA=0.99
def _circulant_window_vec(N,i_1based,P):
 v=np.zeros(N);idx=(np.arange(i_1based-1-P,i_1based+P)%N);v[idx]=1;return v
def Cu_at(u,node,idx,P_=P):
 row=u[idx];vec=_circulant_window_vec(u.shape[1],node+1,P_);return np.dot(row-row[node],vec)
def Cv_at(v,node,idx,P_=P):
 row=v[idx];vec=_circulant_window_vec(v.shape[1],node+1,P_);return np.dot(row-row[node],vec)
def Lu_at(u,v,node,idx):
 u0=u[idx,node];v0=v[idx,node];return u0-(u0**3)/3.0-v0
def Lv_at(u,node,idx,alpha=ALPHA):
 u0=u[idx,node];return u0+alpha
def plot_fhn_nullclines(ax):
 u=np.linspace(UMIN,UMAX,N_SAMPLES);v=u-(u**3)/3.0+I;ax.plot(u,v,color='black',lw=LINE_WIDTH,ls=(0,_DASH),alpha=NULL_ALPHA);ax.axvline(-ALPHA,color='black',lw=LINE_WIDTH,ls=':',alpha=NULL_ALPHA)
def plot_limit_cycle(ax,csv_path=LC_PATH):
 lc=np.genfromtxt(csv_path,delimiter=',',dtype=float);ax.plot(lc[:,0],lc[:,1],LC_STYLE,lw=LC_LINEWIDTH,alpha=LC_ALPHA,color=LC_COLOR)
def make_cmap():
 return LinearSegmentedColormap.from_list('cmap',[(0,'blue'),(0.45,'white'),(0.55,'white'),(1,'red')])
def add_spacetime_inset(ax,u,ia,ib,node,ia_core=None,ib_core=None,nodes=50,rect_color='green',rect_alpha=0.9,rect_lw=1.8,width='80%',height='25%',loc='upper right'):
 d=u.T;iaw,ibw=int(ia),int(ib);data=d[:,iaw:ibw+1];cmap=make_cmap();ins=inset_axes(ax,width=width,height=height,loc=loc,borderpad=0.5);ext=(iaw,ibw+1,0,nodes-1);ins.imshow(data,origin='lower',aspect='auto',cmap=cmap,vmin=-3.0,vmax=1.5,extent=ext,interpolation='none');ins.set_xlim(iaw,ibw+1);ins.set_ylim(0,nodes-1);ia_core=ia_core or iaw;ib_core=ib_core or ibw;ins.set_xticks([ia_core,ib_core+1]);ins.set_xticklabels([r'$t_0$',r'$t_1$'],fontsize=18);ins.set_yticks([0,nodes-1]);ins.set_yticklabels([1,nodes],fontsize=14);y0=node-0.5;rect=Rectangle((ia_core,y0),(ib_core+1-ia_core),1.0,fill=False,edgecolor=rect_color,lw=rect_lw,alpha=rect_alpha);ins.add_patch(rect);return ins
def plot_node_vectors(ax,x,y,vecs,colors):
 for(vx,vy),c in zip(vecs,colors):end=(x+ARROW_BODY_SCALE*vx,y+ARROW_BODY_SCALE*vy);arr=FancyArrowPatch((x,y),end,arrowstyle='->',mutation_scale=ARROW_HEAD_SCALE,lw=ARROW_LW,color=c,alpha=ARROW_ALPHA);arr.set_zorder(ARROW_Z);ax.add_patch(arr)
CLEAN_NOISE_CSV='clean_noise_subset.csv';_NOISE_CSV_ARR=None
def _load_clean_noise_csv():
 global _NOISE_CSV_ARR
 if _NOISE_CSV_ARR is None:
  arr=np.loadtxt(CLEAN_NOISE_CSV,delimiter=',',skiprows=1)
  idxs=arr[:,0].astype(int);noise=arr[:,2:]
  _NOISE_CSV_ARR=(idxs,noise)
 return _NOISE_CSV_ARR
def noise_at(node,abs_idx):
 idxs,noise=_load_clean_noise_csv();
 if abs_idx<=idxs[0]:i=0
 elif abs_idx>=idxs[-1]:i=-1
 else:i=np.searchsorted(idxs,abs_idx);i=min(i,len(idxs)-1)
 return noise[i,node]
'''
Path('figures_lib.py').write_text(figlib_code)

In [ ]:
import numpy as np,matplotlib.pyplot as plt,figures_lib as figlib
def load_csv_matrix(p):
 a=np.loadtxt(p,delimiter=',',skiprows=1)
 idx=a[:,0].astype(int);t=a[:,1].astype(float);d=a[:,2:2+NODES].astype(float);return idx,t,d
def to_abs_index(t):return int(round(t*1000))+START_OFFSET
def pick_row_index(idxs,target):i0=int(idxs[0]);i1=int(idxs[-1]);t=min(max(target,i0),i1);return int(t-i0)
def plot_panel(ax,u,v,cfg,is_bottom=False):
 node,t0,t1,dt0,dt1=cfg['node'],cfg['t0'],cfg['t1'],cfg['dt0'],cfg['dt1']
 idx_u,_,u_m=u;idx_v,_,v_m=v
 ia_abs,ib_abs,iai_abs,ibi_abs=[to_abs_index(x) for x in (t0,t1,t0-dt0,t1+dt1)]
 ia,ib,iai,ibi=[pick_row_index(idx_u,i) for i in (ia_abs,ib_abs,iai_abs,ibi_abs)]
 ia,ib=sorted((ia,ib));iai,ibi=sorted((iai,ibi));iai=max(0,min(iai,ia));ibi=min(max(ib,ibi),u_m.shape[0]-1)
 ax.set_xlim(-2.5,2.0);ax.set_ylim(-1.0,2.1);ax.set_aspect('equal');ax.grid(False)
 if is_bottom:ax.set_xlabel(r'$u$')
 else:ax.set_xlabel('');ax.tick_params(labelbottom=False)
 ax.set_ylabel(r'$v$',rotation=0)
 if cfg.get('show_nullclines',True):figlib.plot_fhn_nullclines(ax)
 if cfg.get('show_limit_cycle',True):figlib.plot_limit_cycle(ax,csv_path=LC_PATH)
 N=u_m.shape[1];mask=np.ones(N,bool);mask[node]=False
 ax.scatter(u_m[ia,:][mask],v_m[ia,:][mask],s=16,c='blue',alpha=0.5)
 ax.scatter(u_m[ib,:][mask],v_m[ib,:][mask],s=16,c='green',alpha=0.5)
 x0,y0=u_m[ia,node],v_m[ia,node];x1,y1=u_m[ib,node],v_m[ib,node]
 ax.scatter([x0],[y0],s=40,c='red',alpha=0.9);ax.scatter([x1],[y1],s=40,c='red',alpha=0.9)
 ax.text(x0+0.04,y0-0.16,r'$t_0$',color='blue',fontsize=18)
 ax.text(x1+0.04,y1-0.21,r'$t_1$',color='green',fontsize=18)
 Cu0,Cv0=figlib.Cu_at(u_m,node,ia),figlib.Cv_at(v_m,node,ia)
 Lu0,Lv0=figlib.Lu_at(u_m,v_m,node,ia),figlib.Lv_at(u_m,node,ia)
 Cu1,Cv1=figlib.Cu_at(u_m,node,ib),figlib.Cv_at(v_m,node,ib)
 Lu1,Lv1=figlib.Lu_at(u_m,v_m,node,ib),figlib.Lv_at(u_m,node,ib)
 s0,s1=figlib.noise_at(node,int(idx_u[ia])),figlib.noise_at(node,int(idx_u[ib]))
 def unit(v):n=np.linalg.norm(v);return v/n if n>0 else v
 AL=0.4;Cs0, Ls0,Ns0=[unit(np.array(v,float))*AL for v in ([Cu0,Cv0],[Lu0,Lv0],[0.0,s0])];Cs1,Ls1,Ns1=[unit(np.array(v,float))*AL for v in ([Cu1,Cv1],[Lu1,Lv1],[0.0,s1])]
 figlib.plot_node_vectors(ax,x0,y0,np.array([Cs0,Ls0,Ns0]),['hotpink','blueviolet','teal'])
 figlib.plot_node_vectors(ax,x1,y1,np.array([Cs1,Ls1,Ns1]),['hotpink','blueviolet','teal'])
 figlib.add_spacetime_inset(ax,u_m,iai,ibi,node,ia_core=ia,ib_core=ib,nodes=NODES,rect_color='black',rect_alpha=0.7,rect_lw=0.5,width='80%',height='25%',loc='upper right')
 if cfg.get('title'):ax.text(0.02,0.98,cfg['title'],transform=ax.transAxes,ha='left',va='top',fontsize=20,fontweight='bold')

In [ ]:
missing=[p for p in[U_CSV_A,V_CSV_A,U_CSV_B,V_CSV_B,LC_PATH,NOISE_CSV]if not Path(p).exists()]
print('Missing files:' if missing else 'All present ✓',missing)

In [ ]:
uA=load_csv_matrix(U_CSV_A);vA=load_csv_matrix(V_CSV_A);uB=load_csv_matrix(U_CSV_B);vB=load_csv_matrix(V_CSV_B)
fig,ax=plt.subplots(2,1,figsize=(6.5,7.5),dpi=300)
plot_panel(ax[0],uA,vA,PANEL_A)
plot_panel(ax[1],uB,vB,PANEL_B,True)
plt.tight_layout()
p1=FIG_DIR/'fig01_2sets_from_csv_noiseCSV.png';p2=FIG_DIR/'fig01_2sets_from_csv_noiseCSV.pdf'
plt.savefig(p1,bbox_inches='tight',dpi=700);plt.savefig(p2,bbox_inches='tight')
print('Saved',p1,p2)